In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/tabular-playground-series-mar-2021/sample_submission.csv
/kaggle/input/tabular-playground-series-mar-2021/train.csv
/kaggle/input/tabular-playground-series-mar-2021/test.csv


In [2]:
train = pd.read_csv('/kaggle/input/tabular-playground-series-mar-2021/train.csv')
test = pd.read_csv('/kaggle/input/tabular-playground-series-mar-2021/test.csv')

In [3]:
categorical_cols = ['cat'+str(i) for i in range(19)]
continous_cols = ['cont'+str(i) for i in range(11)]


In [4]:
cols=categorical_cols+continous_cols
train_objs_num = len(train)
dataset = pd.concat(objs=[train[cols], test[cols]], axis=0)
dataset_preprocessed = pd.get_dummies(dataset,columns=categorical_cols)
train_preprocessed = dataset_preprocessed[:train_objs_num]
test_preprocessed = dataset_preprocessed[train_objs_num:]

In [5]:
train_preprocessed.head()

,cont0,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,...,cat16_C,cat16_D,cat17_A,cat17_B,cat17_C,cat17_D,cat18_A,cat18_B,cat18_C,cat18_D
0,0.629858,0.855349,0.759439,0.795549,0.681917,0.621672,0.592184,0.791921,0.815254,0.965006,...,0,1,0,0,0,1,0,1,0,0
1,0.370727,0.328929,0.386385,0.541366,0.388982,0.357778,0.600044,0.408701,0.399353,0.927406,...,0,0,0,0,0,1,0,1,0,0
2,0.502272,0.322749,0.343255,0.616352,0.793687,0.552877,0.352113,0.388835,0.412303,0.292696,...,0,1,0,0,0,1,0,1,0,0
3,0.934242,0.707663,0.831147,0.807807,0.800032,0.619147,0.221789,0.897617,0.633669,0.760318,...,0,1,0,0,0,1,0,1,0,0
4,0.254427,0.274514,0.338818,0.277308,0.610578,0.128291,0.578764,0.279167,0.351103,0.357084,...,0,0,0,0,0,1,0,1,0,0


<h3>Denoising Auto Encoder</h3>

In [6]:
from tensorflow.keras.layers import Input,Dense,Dropout
from tensorflow.keras import Model
from  tensorflow.keras.regularizers import l2
import tensorflow as tf

In [7]:
def DAutoEnc():
    # denoising autoencoder
    inputs = Input((11,))
    x = Dense(1500, activation='relu')(inputs) # 1500 original
    x = Dense(1500, activation='relu', name="feature")(x) # 1500 original
    x = Dense(1500, activation='relu')(x) # 1500 original
    outputs = Dense(11, activation='relu')(x)
    model = Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer='adam', loss='mse')

    return model

In [8]:
alldata = pd.concat([train[continous_cols],test[continous_cols]],axis=0)
print(alldata.shape)
autoencoder = DAutoEnc()
autoencoder.fit(alldata[continous_cols], alldata[continous_cols],
                    epochs=10,
                    batch_size=256,
                    shuffle=True
                    )

(500000, 11)
Epoch 1/30
3907/3907 [==============================] - 12s 3ms/step - loss: 0.0360
Epoch 2/30
3907/3907 [==============================] - 11s 3ms/step - loss: 0.0278
Epoch 3/30
3907/3907 [==============================] - 11s 3ms/step - loss: 0.0279
Epoch 4/30
3907/3907 [==============================] - 11s 3ms/step - loss: 8.6961e-05
Epoch 5/30
3907/3907 [==============================] - 11s 3ms/step - loss: 1.9588e-05
Epoch 6/30
3907/3907 [==============================] - 11s 3ms/step - loss: 1.9552e-05
Epoch 7/30
3907/3907 [==============================] - 11s 3ms/step - loss: 1.6698e-05
Epoch 8/30
3907/3907 [==============================] - 11s 3ms/step - loss: 1.5491e-05
Epoch 9/30
3907/3907 [==============================] - 11s 3ms/step - loss: 1.4132e-05
Epoch 10/30
3907/3907 [==============================] - 11s 3ms/step - loss: 1.2897e-05
Epoch 11/30
3907/3907 [==============================] - 11s 3ms/step - loss: 1.1445e-05
Epoch 12/30
3907/3907 [======

In [9]:
test_denoised = test_preprocessed.copy()
test_denoised[continous_cols] = autoencoder.predict(test_denoised[continous_cols])
train_denoised = train_preprocessed.copy()
train_denoised[continous_cols] = autoencoder.predict(train_denoised[continous_cols])

In [10]:
train_denoised['target'] = train.target

In [11]:
X_train = train_denoised.drop(['target'], axis=1)
y_train = train_denoised.target
X_test = test_denoised


In [23]:
params={'learning_rate': 0.03,
 'metric': 'auc',
 'n_estimators': 621,
 'num_leaves': 20067,
 'max_depth': 27,
 'reg_alpha': 9.630576598001266,
 'reg_lambda': 2.346945113164939,
 'colsample_bytree': 0.29858836720777177,
 'subsample ': 0.6267448547447422,
 'cat_smooth': 20.45469933468782}



In [24]:
import random
from sklearn.metrics import mean_squared_error,roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold,StratifiedKFold
from lightgbm import LGBMClassifier




preds = np.zeros(test.shape[0])        
kf = StratifiedKFold(n_splits=10,random_state=48,shuffle=True)                  
auc=[]   # list contains AUC for each fold  
n=0   
for trn_idx, test_idx in kf.split(X_train,train['target']):
    X_tr,X_val=X_train.iloc[trn_idx],X_train.iloc[test_idx]
    y_tr,y_val=train['target'].iloc[trn_idx],train['target'].iloc[test_idx]
    model = LGBMClassifier(**params) 
    model.fit(X_tr,y_tr,eval_set=[(X_val,y_val)],early_stopping_rounds=100,verbose=False) 
    preds+=model.predict_proba(X_test)[:, 1]/kf.n_splits 
    auc.append(roc_auc_score(y_val, model.predict_proba(X_val)[:, 1])) 
    print(n+1,auc[n])                                                                                       
    n+=1

1 0.8965264503842034
[LightGBM] [Warning] subsample is set=1.0, subsample= will be ignored. Current value: subsample=1.0
[LightGBM] [Warning] Unknown parameter: 0.6267448547447422
2 0.8934327466269268
[LightGBM] [Warning] subsample is set=1.0, subsample= will be ignored. Current value: subsample=1.0
[LightGBM] [Warning] Unknown parameter: 0.6267448547447422
3 0.8980712593629016
[LightGBM] [Warning] subsample is set=1.0, subsample= will be ignored. Current value: subsample=1.0
[LightGBM] [Warning] Unknown parameter: 0.6267448547447422
4 0.8953157810870424
[LightGBM] [Warning] subsample is set=1.0, subsample= will be ignored. Current value: subsample=1.0
[LightGBM] [Warning] Unknown parameter: 0.6267448547447422
5 0.8961847040389226
[LightGBM] [Warning] subsample is set=1.0, subsample= will be ignored. Current value: subsample=1.0
[LightGBM] [Warning] Unknown parameter: 0.6267448547447422
6 0.8942436124168233
[LightGBM] [Warning] subsample is set=1.0, subsample= will be ignored. Current 

In [25]:
np.mean(auc)

0.8960266844378516

In [26]:
sub = pd.read_csv('/kaggle/input/tabular-playground-series-mar-2021/sample_submission.csv')
sub.head()
sub.shape

(200000, 2)

In [27]:
sub['target']=preds
sub.to_csv('submission.csv', index=False)